In [1]:
import nbaScraper as ns
#import actNetScraper as ans
import actNetApi as ans
import nbApi as nbapi
from datetime import datetime, timedelta
# -----------------------------------------------
# -------------------------------- PARAMS
leagues = None  # None or list ['nba', 'nhl', 'nfl', 'mlb', 'wnba'] # NONE looks for all sports
specified =  []  ####  [specific prop] or [] for all props, *****only works with a single league in leagues

# day adjustment from today (date of running script), negative = dates into the past
dayJump = 0
# date can be a list of dates if multiple need scraping 'YYYY-MM-DD'
# default is to only pull today or today + dayJump
dates = [(datetime.today() + timedelta(days=dayJump)).strftime('%Y-%m-%d')]
#dates = ['2025-08-18']#, '2025-08-17']#', '2025-07-28']#, '2025-01-07']

database_export = True  # add all scrapes to database
store_locally = True    # add all scrapes to class variables
season_int = 2025 # int will be the final year of the schedule season
season_str = '2024-25'
season_start_date = '2024-10-22' #'MM/DD/YYY'
season_type = 'Playoffs' # ['Regular+Season', 'PlayIn', 'Playoffs']
per_mode = 'Totals' #['Totals', 'PerGame']
# ------------------------------------------------

# nba website and basketball referenece scrapers
scraper = ns.scraper(
    browser_path = '..\\..\\browser\\geckodriver.exe',
    database_export = database_export, 
    store_locally = store_locally,
    pymysql_conn_str =  None
)
# assigns the date of the last time code executed as today
today = scraper.meta_data['today_dt']

# looks up the actual date for the last regular season game date. this will be used to grab the data for the completed games on the date
nbaApi = nbapi.nbaApi(
    browser_path = '..\\..\\browser\\geckodriver.exe',
    database_export = database_export, 
    store_locally = store_locally,
    pymysql_conn_str =  None
)
a = nbaApi.get_last_game_date(season = season_str)
run_date = nbaApi.last_game_date
dateRange = [
    run_date, run_date
]

#prop_scraper = ans.actNetScraper(
prop_scraper = ans.actNetApi(    
    browser_path = '..\\..\\browser\\geckodriver.exe',
    dates = dates,
    leagues = leagues,
    database_export = database_export, 
    store_locally = store_locally,
    config_path = '..\\..\\..\\..\\Notes-General\\config.txt',
    second_run = False
)

# turn to False if issues loading new players
#prop_scraper.update_players = False

# update the league list to only ones with games today
#leagues = prop_scraper.check_for_league_games(date_check = None, update_class_leagues_var = True)
print('scraping for', prop_scraper.leagues, 'on', prop_scraper.dates)

Last day with games played: 2025-06-22
scraping for ['mlb', 'wnba'] on ['2025-08-20']


In [2]:
# SCRAPE PROPS
print(today, 'run date...\n for', dates, 'and', prop_scraper.leagues)
############## these 2 lines were used w. old selenium setup in actNetScraper.py
#prop_scraper.scrape(sleep_secs = 3, specific_props = specified,leagues_override = prop_scraper.leagues,an_state_code = 'BC')
#prop_scraper.processScrapes(remove_dups = True,specific_props = specified)
###############
prop_scraper.scrape(
    sleep_secs = 3, 
    specific_props = specified,
    leagues_override = prop_scraper.leagues,
    an_state_code = 'BC'
)

prop_scraper.processScrapes(
    remove_dups = True,
    specific_props = specified
) 

if prop_scraper.scrape_error_flag:
    print(prop_scraper.scrape_errors)
    #prop_scraper.tryMissingProps()


2025-08-20 run date...
 for ['2025-08-20'] and ['mlb', 'wnba']
scraping mlb ...
original rows:  (2982, 20)
after dups removed:  (2373, 20)
['Parker Messick' 'Will Banfield']
mlb odds data loaded...
prop          dbl  er    h   hr   k   o  rbi   rs  single  stlb   tb
propId count  295  27  296  293  28  26  283  296     295   251  283
scraping wnba ...
original rows:  (45, 20)
after dups removed:  (45, 20)
[]
wnba odds data loaded...
prop          ast  pa  pr  pra  pts  ra  reb  threes
propId count    4   4   8    5    8   3    8       5


In [19]:
# SCRAPE BREF TEAM MISC
print(today, 'run date...\n')
#teams = ['GSW','DEN','POR','SAC','TOR','DAL','PHO','CHI','LAL','HOU','MIA','MEM','DET','MIL','NOP','MIN','CLE','OKC','LAC','BRK','SAS','NYK','WAS','CHO','UTA','IND','BOS','PHI','ATL','ORL']
scraper.get_bref_pos_estimates(
        base_url = 'https://www.basketball-reference.com/teams/{team}/{season}.html#pbp', 
        today_date = today,
        season = season_int,
        database_table = 'brefmisc',
        team_overrides = None
)
### CAN DELETE AFTER CONFIRMING ERROR RETRIES IN FUNCTION WORK AS EXPECTED
missing_teams = scraper.scrape_errors['brefmisc']['url']
if len(missing_teams):
        print('missing:', missing_teams)
        for i in missing_teams:
                scraper.get_bref_pos_estimates(
                        base_url = 'https://www.basketball-reference.com/teams/{team}/{season}.html#pbp', 
                        today_date = today,
                        season = season_int,
                       database_table = 'brefmisc',
                        team_overrides = [i[0]]
               )


2025-06-22 run date...

bref player pos estimates scraped, 30 teams...


In [20]:
# NBA API hits
print(today, 'run date...\n')
# --- TEAM --- #
"""nbaApi.request_nba_team_playtype_data(
            season = season_str, #'YYYY-YY' 
            play_type = [
                'Isolation', 'Transition','PRBallHandler','PRRollman', 'Postup', 'Spotup', 
                'Handoff', 'Cut', 'OffScreen', 'OffRebound', 'Misc'
            ], 
            lid='00',
            per_mode = 'PerGame', #['Totals', 'PerGame'] 
            season_type = season_type, #['Regular+Season', 'PlayIn', 'Playoffs']
            sob = ['offensive', 'defensive'],
            sleep_time = 2,
            database_table = 'statsteamplaytypes'
        )
"""
print(today, 'run date...\n')
nbaApi.request_nba_team_shotzone_data(
    season = season_str, #'YYYY-YY'
    start_date = run_date, # will force to pd.datetime.date()
    end_date = run_date, 
    per_mode = per_mode, #['Totals', 'PerGame'] 
    season_type = season_type, #['Regular+Season', 'PlayIn', 'Playoffs']
    distance_type = 'By+Zone', # ['8ft+Range', '5ft+Range','By+Zone']
    sob = ['Base', 'Opponent'], #['Base', 'Opponent']  base= teams offense, opponent = teams defense
    database_table = 'statsteamshotzones'     
)

# PER GAME
nbaApi.request_nba_team_stats(
    season = season_str, #'YYYY-YY'
    start_date = season_start_date, # will force to pd.datetime.date()
    end_date = run_date, 
    per_mode = 'PerGame', #['Totals', 'PerGame'] 
    season_type = season_type, #['Regular+Season', 'PlayIn', 'Playoffs']
    measure_type = ['Base', 'Advanced', 'Opponent'], #['Base', 'Advanced', 'Opponent']  
    database_table = 'statsteam'
)
# TOTALS
nbaApi.request_nba_team_stats(
    season = season_str, #'YYYY-YY'
    start_date = run_date, # will force to pd.datetime.date()
    end_date = run_date, 
    per_mode = 'Totals', #['Totals', 'PerGame'] 
    season_type = season_type, #['Regular+Season', 'PlayIn', 'Playoffs']
    measure_type = ['Base', 'Advanced', 'Opponent'], #['Base', 'Advanced', 'Opponent']  
    database_table = 'statsteamtotals'
)

# --- PLAYER --- #
print(today, 'run date...\n')
"""nbaApi.request_nba_player_playtype_data(
            season = season_str, #'YYYY-YY' 
            play_type = [
                'Isolation', 'Transition','PRBallHandler','PRRollman', 'Postup', 'Spotup', 
                'Handoff', 'Cut', 'OffScreen', 'OffRebound', 'Misc'
            ], 
            lid='00',
            per_mode = 'PerGame', #['Totals', 'PerGame']
            season_type = season_type, #['Regular+Season', 'PlayIn', 'Playoffs']
            sob = ['offensive'], #'offensive','defensive'],
            sleep_time = 2,
            database_table = 'statsplayerplaytypes'
)
"""

print(today, 'run date...\n')
nbaApi.request_nba_player_shotzone_data(
            season = season_str, #'YYYY-YY'
            start_date = run_date, # will force to pd.datetime.date()
            end_date = run_date, 
            per_mode = per_mode, #['Totals', 'PerGame'] 
            season_type = season_type, #['Regular+Season', 'PlayIn', 'Playoffs']
            distance_type = 'By+Zone', # ['8ft+Range', '5ft+Range','By+Zone'],
            sob = ['Base'], #['Base', 'Opponent']  base= teams offense, opponent = teams defense
            database_table = 'statsplayershotzones'        
)

# Passing
nbaApi.request_nba_player_tracking(
        season = season_str, #'YYYY-YY'
        start_date =run_date, # will force to pd.datetime.date()
        end_date = run_date, 
        per_mode = per_mode, #['Totals', 'PerGame'] 
        season_type = season_type, #['Regular+Season', 'PlayIn', 'Playoffs']
        measure_type = 'Passing', #['Passing','Rebounding','Drives','Possessions','Efficiency','PostTouch', 'ElbowTouch', 'PaintTouch']  
        database_table = 'statsplayerpassing'      
)

# Rebounding
nbaApi.request_nba_player_tracking(
        season = season_str, #'YYYY-YY'
        start_date =run_date, # will force to pd.datetime.date()
        end_date = run_date, 
        per_mode = per_mode, #['Totals', 'PerGame'] 
        season_type = season_type, #['Regular+Season', 'PlayIn', 'Playoffs']
        measure_type = 'Rebounding', #['Passing','Rebounding','Drives','Possessions','Efficiency']  
        database_table = 'statsplayerrebounding'      
)

# add these measure types - 
#### 'Drives', 'Possessions', 'Efficiency' 
# maybe add these - 'PostTouch', 'ElbowTouch', 'PaintTouch'
#nbaApi.request_nba_player_tracking()

2025-06-22 run date...

2025-06-22 run date...

nba team shot zone retrieved 2 offensive,  2 defensive loaded...
nba team stats PerGame retrieved (16, 65) loaded...
nba team stats Totals retrieved (2, 65) loaded...
2025-06-22 run date...

2025-06-22 run date...

nba player shot zone retrieved 27 offensive loaded...
nba player Passing retrieved 27 loaded...
nba player Rebounding retrieved 22 loaded...


# old browser scrape for nba data

# loading misscraped passing and rebound date 

In [7]:
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import leaguegamefinder

In [1]:
cstring = scraper.pymysql_conn_str
# qery db to get dates of messed up dates
df = pd.read_sql_query(
    #sql = """SELECT date, COUNT(PLAYER_NAME) n FROM statsplayershotzones GROUP BY date HAVING date > '2024-10-20' and n = 50 ORDER BY date;""",
    sql = "SELECT DISTINCT date FROM statsteamtotals WHERE date > '2023-10-20' AND date < '2024-07-01';",
    con = cstring
)
#dates = df[df['n'] == 50]['date'].tolist()
dates = df['date'].tolist()


NameError: name 'scraper' is not defined

In [94]:
db = [pd.to_datetime(i).date() for i in dates]
actual = dates_reg_2324 + dates_post_2324
np.setdiff1d(actual, db)

array([], dtype=object)

In [99]:
 # Fetch all games for the current season
seasons = '2024-25'
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=seasons, league_id_nullable='00')
games2425 = gamefinder.get_data_frames()[0]
games2425.loc[:,'GAME_DATE'] = pd.to_datetime(games2425['GAME_DATE'])
games2425 = games2425[games2425['SEASON_ID'].astype(str).str.startswith(('2', '4', '5'))]
dates_reg_2425 = games2425[games2425['SEASON_ID'].astype(str).str.startswith(('2'))]['GAME_DATE'].unique().tolist()
dates_reg_2425 = [i.date() for i in dates_reg_2425]
dates_reg_2425.sort()
#dates_post_2425 = games2425[games2425['SEASON_ID'].astype(str).str.startswith(('4', '5'))]['GAME_DATE'].unique().tolist()
#dates_post_2425 = [i.date() for i in dates_post_2324]
#dates_post_2425.sort()


In [ ]:
### HISTORICAL LOADERS
####
for i in []:
    print(i)
    d = pd.to_datetime(i)
    dateRange = [d, d]
    scraper.get_nba_team_shotzone_data(            
        base_url = 'https://www.nba.com/stats/teams/{sideOfBall}?DistanceRange=By+Zone&SeasonType={type}&DateFrom={startDate}&DateTo={endDate}&PerMode={perMode}&Season={season}', 
        sides = {'offensive':'shooting', 'defensive':'opponent-shooting'},
        season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
        perMode = 'Totals', #[Totals, PerMode]
        database_table = 'statsteamshotzones',
        dateRange = dateRange,
        season = '2023-24'
    )


for i in []:
    print(i)
    d = pd.to_datetime(i)
    dateRange = [d, d]
    scraper.get_nba_player_shotzone_data(
        #base_url = 'https://www.nba.com/stats/players/shooting?DistanceRange=By+Zone&LastNGames={lastNgames}&SeasonType={type}&DateTo={endDate}', 
        base_url = 'https://www.nba.com/stats/players/shooting?DistanceRange=By+Zone&SeasonType={type}&DateFrom={startDate}&DateTo={endDate}&PerMode=Totals&Season=2023-24',  
        season_type = 'PlayIn',  # ['Regular+Season', 'PlayIn', 'Playoffs']
        lastNgames = 0,
        database_table = 'statsplayershotzones',
        dateRange = dateRange
    )


for i in []:
    print(i)
    d = pd.to_datetime(i)
    scraper.get_nba_player_passing_data(
        base_url = 'https://www.nba.com/stats/players/passing?DateFrom={d1}&DateTo={d2}&PerMode=Totals&SeasonType={type}&Season=2023-24', 
        run_date = d,
        season_type = 'Playoffs',  # ['Regular+Season', 'PlayIn', 'Playoffs']
        database_table = 'statsplayerpassing'
    )

#
for i in []:
    print(i)
    d = pd.to_datetime(i)
    scraper.get_nba_player_rebounding_data(
        base_url = 'https://www.nba.com/stats/players/rebounding?DateFrom={d1}&DateTo={d2}&PerMode=Totals&SeasonType={type}&Season=2023-24', 
        run_date = d,
        season_type = 'Playoffs',  # ['Regular+Season', 'PlayIn', 'Playoffs']
        database_table = 'statsplayerrebounding'
    )

p = 'Regular+Season'
for i in dates_reg_2425:
    print(i)
    d = pd.to_datetime(i)
    dateRange = [d, d]  
    # per game
    scraper.get_nba_team_stats(
        base_url = 'https://www.nba.com/stats/teams/{stats}?SeasonType={seasonType}&DateFrom={startDate}&DateTo={endDate}&PerMode={perMode}&Season={season}',
        stats = ['traditional', 'advanced', 'opponent'],
        season_type = p,  # ['Regular+Season', 'PlayIn', 'Playoffs']
        perMode = 'PerGame',  # [Totals, PerGame]
        database_table = 'statsteam',
        dateRange = dateRange,
        season = '2024-25'
    )
    # totals
    scraper.get_nba_team_stats(
        base_url = 'https://www.nba.com/stats/teams/{stats}?SeasonType={seasonType}&DateFrom={startDate}&DateTo={endDate}&PerMode={perMode}&Season={season}',
        stats = ['traditional', 'advanced', 'opponent'],
        season_type = p,  # ['Regular+Season', 'PlayIn', 'Playoffs']
        perMode = 'Totals',  # [Totals, PerGame]
        database_table = 'statsteamtotals',
        dateRange = dateRange,
        season = '2024-25'
    )

# scratch

In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time 